In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
import sys
import os

sys.path.insert(0, "/home/pavithra/projects/Learnings/")
from open_ai_keys import LONGSMITH_LEARNING_API_KEY, GEMINI_LEARNING_API_KEY
# Set the Key as an environment variable.
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LONGSMITH_LEARNING_API_KEY

# Setting up the API key for the gemini model.
os.environ["GOOGLE_API_KEY"] = GEMINI_LEARNING_API_KEY

- 1. Loader
- 2. Split
- 3. Store
- 4. Retrive
- 5. Generate

# Basic learning

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

/home/pavithra/projects/Learnings/genai_learning/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
llm.invoke("what is your name?")

I0000 00:00:1723714751.972793   45342 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


AIMessage(content="I am a large language model, trained by Google. \n\nI don't have a name like a person does. You can call me Bard, or simply refer to me as Google AI. \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-3f1891df-5ad5-401b-914f-cfaa2388f4ea-0', usage_metadata={'input_tokens': 6, 'output_tokens': 40, 'total_tokens': 46})

In [5]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore programmer. \n", response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-63e06e2c-ee3f-46a7-818d-5de4c70eeab7-0', usage_metadata={'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 26})

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe das Programmieren. \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-951ae1b4-fdba-47d5-b2c2-5dced5a18764-0', usage_metadata={'input_tokens': 16, 'output_tokens': 7, 'total_tokens': 23})

# 2 | Document loader

In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Read all the documents in the given folder. TODO: Make the folder configurable and get it from the config file.
loader = PyPDFDirectoryLoader("/home/pavithra/projects/Learnings/LongChain/data")
docs = loader.load()
docs

[Document(metadata={'source': '/home/pavithra/projects/Learnings/LongChain/data/2_RM - Classification of Stochastic Processes.pdf', 'page': 0}, page_content='  \n \n \n \n \n \n \n \n \n \n \n \nMDS5 204/ Segment  1 \n \n CLASSIFICATION OF \nSTOCHASTIC PROCESSES  \n \n \n \n \n \n \n'),
 Document(metadata={'source': '/home/pavithra/projects/Learnings/LongChain/data/2_RM - Classification of Stochastic Processes.pdf', 'page': 1}, page_content='CLASSIFICATION  OF STOCHASTIC  PROCESSES   \n©COPYRIGHT 2022 (VER. 1.0), ALL RIGHTS RESERVED. MANIPAL ACADEMY OF HIGHER EDUCATION                                                   2/8 \nTable of Contents  \n1. Stochastic Processes  4 \n2. Classification of Stochastic Process  4 \n2.1 Parameter Space  5 \n2.2 State Space  5 \n2.3 Discrete and Continuous Property  5 \n3. Summary  8 \n  \n \n \n \n \n  '),
 Document(metadata={'source': '/home/pavithra/projects/Learnings/LongChain/data/2_RM - Classification of Stochastic Processes.pdf', 'page': 2}, pag

In [8]:
print(docs[0].page_content[:500])

  
 
 
 
 
 
 
 
 
 
 
 
MDS5 204/ Segment  1 
 
 CLASSIFICATION OF 
STOCHASTIC PROCESSES  
 
 
 
 
 
 



In [9]:
for document in docs:
    print(len(document.page_content)) # The max is 2.5l length

104
389
631
1659
1973
1551
2140
1380
97
431
778
2016
2139
2236
1856
1369
1973
2084
591


# 2 | Split the document 

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=400,
                                               add_start_index=True)
splits: list = text_splitter.split_documents(docs)

In [11]:
print(f"Length of the documents: {len(splits)}")

Length of the documents: 41


In [12]:
len(splits[8].page_content)

747

In [13]:
splits[8].metadata

{'source': '/home/pavithra/projects/Learnings/LongChain/data/2_RM - Classification of Stochastic Processes.pdf',
 'page': 4,
 'start_index': 1224}

In [14]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

vectorstore = Chroma.from_documents(collection_name="user_document",
                                    documents=splits,
                                    embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
                                    )


# 4 | Retriver


In [17]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("What is stocastic process?")


In [18]:
retrieved_docs

[Document(metadata={'page': 8, 'source': '/home/pavithra/projects/Learnings/LongChain/data/1_RM - Introduction to Stochastic Processes.pdf', 'start_index': 0}, page_content='INTRODUCTION  TO STOCHASTIC  PROCESSES   \n©COPYRIGHT 202 3 (VER. 1.0), ALL RIGHTS RESERVED. MANIPAL ACADEMY OF HIGHER EDUCATION                                                   9/11 \n \nFigure 3: Observation of Stochastic Process  \nThe graph shows five sample paths of a stochastic process {Xt} where t≥0. So, 𝑋𝑡(𝑤1), \n𝑋𝑡(𝑤2)…, these are  nothing but X1, X2, X3, X4, X5, all are different sample paths or sample \nfunctions of the stochastic process Xt. The average or the expected value 𝐸{X𝑡} of the \nprocess is also drawn  (in yellow) , and that it is not a sample path; it is a deterministic \nfunction of 𝑡.  \nA stochastic process X=(Xt|t∈I) is a collection of random variables Xt. In this, w is \nsuppressed and X(t) or Xt is used to denote a random process. Taking values in some (real -\nvalued) set S,Xt(𝜔)∈S, a

In [19]:
print(retrieved_docs[0].page_content)


INTRODUCTION  TO STOCHASTIC  PROCESSES   
©COPYRIGHT 202 3 (VER. 1.0), ALL RIGHTS RESERVED. MANIPAL ACADEMY OF HIGHER EDUCATION                                                   9/11 
 
Figure 3: Observation of Stochastic Process  
The graph shows five sample paths of a stochastic process {Xt} where t≥0. So, 𝑋𝑡(𝑤1), 
𝑋𝑡(𝑤2)…, these are  nothing but X1, X2, X3, X4, X5, all are different sample paths or sample 
functions of the stochastic process Xt. The average or the expected value 𝐸{X𝑡} of the 
process is also drawn  (in yellow) , and that it is not a sample path; it is a deterministic 
function of 𝑡.  
A stochastic process X=(Xt|t∈I) is a collection of random variables Xt. In this, w is 
suppressed and X(t) or Xt is used to denote a random process. Taking values in some (real -
valued) set S,Xt(𝜔)∈S, and indexed by a real -valued (time) parameter t  ∈I. Here, I is 
referred to  as the parameter space and S is referred to  as the sample space.  Stochastic


# 5 | Generator

In [32]:
# Retrieve and generate using the relevant snippets of the blog.
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = hub.pull("rlm/rag-prompt")
print((prompt))
prompt_template: str = """Your name is noso, an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
 If you don't know the answer, just say that you don't know. Explain the answer in maximum 5 sentences. Answer like talking to a close friend.
 \nQuestion: {question} \nContext: {context} \nAnswer:"""

promt = ChatPromptTemplate([
    ("system", "Your name is noso. You are an assistant for question-answering task from the user uploaded document"),
    ("human", prompt_template)
])
retriever = vectorstore.as_retriever()
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Hey man, What are all the classifications stocastic processes?"))

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]
Stochastic processes are classified based on parameter space and state space. Both parameter and state space are further classified based on whether they are discrete or continuous. Therefore, there are four classifications of stochastic processes based on the combination of discrete or continuous parameter/state space. 



In [33]:
print(rag_chain.invoke("What is the history of stochastic processes?"))

This text focuses on defining and classifying stochastic processes, not their history. Therefore, I cannot answer your question. This text explains concepts like sample space, parameter space, and how stochastic processes are visualized. It also differentiates between deterministic and stochastic models. 



- about you file
- study about different RAG methods and retrivers see anything interesting and useable
- front end -- keep it question and answer, already asked questions in the side like chatgpt(store it in a file)
- docker containers
- github readme 